In [ ]:
import requests

app_id = 'd1f4cfd5-2a86-4c76-8593-5f999bf1a02d' 
api_key = 'MLXMoaTgjEozDqGUPJbWBsYQoKCwZ06n'

In [ ]:
url = 'https://api.nytimes.com/svc/archive/v1/2019/1.json?api-key='+api_key
r = requests.get(url)
data = r.json()
data

In [ ]:
import pyjq
copyright = pyjq.all('.copyright', data)
num_docs = pyjq.all('.response .docs | length', data)[0]
print(num_docs)

In [ ]:
jq_query = f'.response .docs [] | {{url: .web_url, snippet: .snippet, headline: .headline .main, date: .pub_date, category: .news_desk}}'
output = pyjq.all(jq_query, data)
output

In [ ]:
len(output)

In [ ]:
output[0]['snippet']

In [ ]:
for key, value in output[0].items():
    print(key)
    print(value)

In [ ]:
business = []
for i in output:
    if i['category'] == 'Business':
        business.append(i)  

In [ ]:
business

In [ ]:
len(business)

In [ ]:
import pandas as pd
df = pd.DataFrame(business)
df.drop_duplicates(keep=False, inplace=True)
df

In [ ]:
df['date'] = df['date'].map(lambda i: str(i)[:10])

In [ ]:
import datetime
df['date'] = df['date'].map(lambda i: datetime.datetime.strptime(i, '%Y-%m-%d'))

In [ ]:
df.info()

In [ ]:
df['text'] = df.snippet + df.headline

In [ ]:
df.drop(columns=['snippet', 'headline', 'category'], inplace=True)

In [ ]:
df.set_index('date', inplace=True)
df.sort_index(axis=0, inplace=True)

In [ ]:
df

In [ ]:
# chaining above codes into one function
import requests
import pyjq
import pandas as pd
import datetime

def monthly_text(year, month):
    app_id = 'd1f4cfd5-2a86-4c76-8593-5f999bf1a02d' 
    api_key = 'MLXMoaTgjEozDqGUPJbWBsYQoKCwZ06n'
    url = 'https://api.nytimes.com/svc/archive/v1/' +str(year)+ '/' +str(month)+ '.json?api-key=' +api_key
    r = requests.get(url)
    data = r.json()
    jq_query = f'.response .docs [] | {{url: .web_url, snippet: .snippet, headline: .headline .main, date: .pub_date, category: .news_desk}}'
    output = pyjq.all(jq_query, data)
    business = []
    for i in output:
        if i['category'] == 'Business':
            business.append(i)
    df = pd.DataFrame(business)
    df.drop_duplicates(keep=False, inplace=True)
    df['date'] = df['date'].map(lambda i: str(i)[:10])
    df['date'] = df['date'].map(lambda i: datetime.datetime.strptime(i, '%Y-%m-%d'))
    df['text'] = df.snippet + df.headline
    df.drop(columns=['snippet', 'headline', 'category'], inplace=True)
    df.sort_values('date', inplace=True)
    return df        

In [ ]:
Jan19_df = monthly_text(2019, 1)
Jan19_df

In [ ]:
import json

d = Jan19_df.to_dict(orient='records')
d

In [ ]:
# scraping each article by url from dictionary

from bs4 import BeautifulSoup as BS


def get_nyt_text(url):
    headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'}
    page = requests.get(url, headers=headers,timeout=5)
    page.status_code
    
    soup = BS(page.content, 'html.parser')
    content = soup.findAll('p', class_ = 'css-18icg9x evys1bk0')
    
    nyt = ''
    for index in range(len(content)):
        nyt += content[index].get_text()
        
    return nyt

In [ ]:
for i in d:
    i.update({'article':get_nyt_text(i['url'])})

In [ ]:
d

In [ ]:
# using Vader for sentiment analysis

from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')
si = SentimentIntensityAnalyzer()

for i in d:
    i.update({'sentiment':si.polarity_scores(i['article'])})

In [ ]:
d

In [ ]:
for i in d:
    i.update({'sentiment_headline':si.polarity_scores(i['text'])})

In [ ]:
d

In [ ]:
d[0]['sentiment_headline']['pos']

In [ ]:
for i in d:
    i.update({'pos_headline': i['sentiment_headline']['pos']})
    i.update({'neg_headline': i['sentiment_headline']['neg']})
    i.update({'neu_headline': i['sentiment_headline']['neu']})
d

In [ ]:
df_sentiment = pd.DataFrame(d)
df_sentiment

In [ ]:
import numpy as np

df_sentiment['article'].replace('', np.nan, inplace=True)
df_sentiment.dropna(axis=0)

In [ ]:
df_sentiment.drop(columns=['article', 'sentiment', 'sentiment_headline', 'url'], inplace=True)
df_sentiment

In [ ]:
daily_sentiment = df_sentiment.groupby('date').mean()
daily_sentiment['net_sentiment'] = daily_sentiment.neu_headline + daily_sentiment.pos_headline - daily_sentiment.neg_headline
daily_sentiment

In [ ]:
# time series analysis on S&P 500 for the month of January 2019

SP500 = pd.read_csv('^GSPC.csv')
SP500

In [ ]:
SP500['Date'] = SP500['Date'].map(lambda i: datetime.datetime.strptime(i, '%Y-%m-%d'))

In [ ]:
# daily_sentiment.merge(SP500, left_index=True, right_on='Date')

In [ ]:
SP500.drop(columns=['High', 'Low', 'Adj Close', 'Volume'], inplace=True)

In [ ]:
SP500['Daily_Change'] = SP500['Close'] / SP500['Open'] -1

In [ ]:
SP500.set_index('Date', inplace=True)

In [ ]:
SP500.drop(columns=['Open', 'Close'], inplace=True)

In [ ]:
%matplotlib inline 
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
import statsmodels.api as sm

SP500.plot(figsize=(12,8), title= 'S&P 500 Daily Performance', fontsize=14)

In [ ]:
SP500

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(SP500, freq=1)
fig = plt.figure()
fig = decomposition.plot()
fig.set_size_inches(15, 8)

In [ ]:
daily_sentiment

In [ ]:
daily_sentiment_diff = daily_sentiment.diff(periods=1 ,axis=0)

In [ ]:
daily_net_sentiment = daily_sentiment.merge(daily_sentiment_diff, left_index=True, right_index=True)
daily_net_sentiment

In [ ]:
daily_net_sentiment['Open_Sentiment'] = daily_net_sentiment['net_sentiment_x'] - daily_net_sentiment['net_sentiment_y']


In [ ]:
daily_net_sentiment['Closing_Sentiment'] = daily_net_sentiment['net_sentiment_x']
daily_net_sentiment['Daily_Change_Sentiment'] = daily_net_sentiment['Closing_Sentiment'] / daily_net_sentiment['Open_Sentiment'] -1

In [ ]:
daily_Change_Net_Sentiment = pd.DataFrame(daily_net_sentiment['Daily_Change_Sentiment'])
daily_Change_Net_Sentiment

In [ ]:
SP500_Sentiment = daily_Change_Net_Sentiment.merge(SP500, left_index=True, right_index=True)
SP500_Sentiment

In [ ]:
SP500_Sentiment.plot(figsize=(12,8), title= 'S&P 500 Daily Performance and Change in NYT Sentiment', fontsize=14)